# Lemmatize CORDIS projects using Spark NLP

In [1]:
from sparknlp.base import *
from sparknlp.annotator import *
import sparknlp
from pyspark.ml import Pipeline
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pathlib import Path

## 1. Read project information and concatenate the `title` and `objective` fields

In [2]:
%%time

# Loading papers table text fields, and concatenating them for lemmatization
#projects = spark.sql("SELECT projectID, title, objective FROM parquet.`/export/ml4ds/IntelComp/Datalake/CORDIS/20220221/new_parquet/projects.parquet`")
projects = spark.read.parquet('/export/ml4ds/IntelComp/Datalake/CORDIS/20220221/new_parquet/projects.parquet')
projects = projects.repartition(numPartitions=1000)

#Concatenate text fields to lemmatize
projects = (
    projects.withColumn("rawtext",F.concat_ws('. ', "title", "objective"))
    .select("projectID","rawtext")
)

print('Number of projects before language filtering:', projects.count())

Number of projects before language filtering: 61163
CPU times: user 25.7 ms, sys: 5.49 ms, total: 31.2 ms
Wall time: 1min 1s


## 2. Filter project abstracts that are not in English Language

In [3]:
%%time

#Pipeline for language detection
documentAssembler = DocumentAssembler() \
    .setInputCol("rawtext") \
    .setOutputCol("document")

languageDetector = LanguageDetectorDL.pretrained() \
    .setInputCols("document") \
    .setOutputCol("language")

pipeline = Pipeline() \
    .setStages([
      documentAssembler,
      languageDetector
    ])

#Apply language detection pipeline
projects = pipeline.fit(projects).transform(projects)
projects = (
    projects.filter(F.col("language.result")[0]=="en")
    .drop("language")
)

print('Number of projects in English:', projects.count())

ld_wiki_tatoeba_cnn_21 download started this may take some time.
Approximate size to download 7.1 MB
[ | ]ld_wiki_tatoeba_cnn_21 download started this may take some time.
Approximate size to download 7.1 MB
Download done! Loading the resource.
[ / ]

[ — ]

[ \ ]

[ | ]

2022-05-04 15:47:48.473516: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-04 15:47:48.554100: I external/org_tensorflow/tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2399870000 Hz


[OK!]


22/05/04 15:47:54 ERROR scheduler.TaskSchedulerImpl: Lost executor 2 on node07.cluster.tsc.uc3m.es: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/05/04 15:47:54 WARN scheduler.TaskSetManager: Lost task 19.0 in stage 7.0 (TID 1068) (node07.cluster.tsc.uc3m.es executor 2): ExecutorLostFailure (executor 2 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/05/04 15:47:54 ERROR scheduler.TaskSchedulerImpl: Lost executor 0 on node01.cluster.tsc.uc3m.es: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/05/04 15:47:54 WARN scheduler.TaskSetManager: Lost task 26.0 in stage 7.0 (TID 1067) (node01.cluster.tsc.uc3m.es executor 0): ExecutorLostFailure (executor 0 exited caused by on

22/05/04 15:47:58 ERROR scheduler.TaskSchedulerImpl: Lost executor 6 on node70.cluster.tsc.uc3m.es: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/05/04 15:47:58 WARN scheduler.TaskSetManager: Lost task 29.0 in stage 7.0 (TID 1097) (node70.cluster.tsc.uc3m.es executor 6): ExecutorLostFailure (executor 6 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/05/04 15:47:58 WARN scheduler.TaskSetManager: Lost task 9.0 in stage 7.0 (TID 1080) (node70.cluster.tsc.uc3m.es executor 6): ExecutorLostFailure (executor 6 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/05/04 15:47:58 WARN scheduler.TaskSetManager: Lost task 20.0 in st

22/05/04 15:49:02 ERROR scheduler.TaskSchedulerImpl: Lost executor 10 on node72.cluster.tsc.uc3m.es: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/05/04 15:49:02 WARN scheduler.TaskSetManager: Lost task 24.2 in stage 7.0 (TID 1115) (node72.cluster.tsc.uc3m.es executor 10): ExecutorLostFailure (executor 10 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/05/04 15:49:02 WARN scheduler.TaskSetManager: Lost task 5.2 in stage 7.0 (TID 1114) (node72.cluster.tsc.uc3m.es executor 10): ExecutorLostFailure (executor 10 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/05/04 15:49:02 WARN scheduler.TaskSetManager: Lost task 33.2 

Py4JJavaError: An error occurred while calling o152.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 15 in stage 7.0 failed 4 times, most recent failure: Lost task 15.3 in stage 7.0 (TID 1120) (node64.cluster.tsc.uc3m.es executor 13): ExecutorLostFailure (executor 13 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2253)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2202)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2201)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(Arr

## 3. Define and Run Lemmatization Pipeline

   - We work on documents created in Subsection 2
   - Sentence Detection and Tokenizer applied to detect tokens
   - Lemmatization is carried out
   - Stopwords are applied
   - Punctuation symbols are removed
   - Result is converted back from Spark NLP annotations to string format

In [17]:
%%time 

#Next, we carry out the lemmatization pipeline

sentenceDetector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

lemmatizer = LemmatizerModel.pretrained() \
    .setInputCols(["token"]) \
    .setOutputCol("lemma")

stopWords = StopWordsCleaner() \
    .setInputCols(["lemma"]) \
    .setOutputCol("cleanlemma")

normalizer = Normalizer() \
    .setInputCols(["cleanlemma"]) \
    .setOutputCol("normalizedlemma") \
    .setLowercase(True) \
    .setCleanupPatterns(["""[^\w\d\s]"""])

finisher = Finisher() \
     .setInputCols(['normalizedlemma'])

pipeline = Pipeline() \
    .setStages([
#      documentAssembler,
      sentenceDetector,
      tokenizer,
      lemmatizer,
      stopWords,
      normalizer,
      finisher
])

#We apply pipeline and recover lemmas as string
projects = pipeline.fit(projects).transform(projects)

udf_back2str = F.udf(lambda x:' '.join(list(x)), StringType() )
projects = (
    projects.withColumn("lemmas",udf_back2str(F.col("finished_normalizedlemma")))
    .drop("rawtext")
    .drop("finished_normalizedlemma")
)

#Show results of validation for n papers
projects.show(n=10, truncate=120, vertical=True)

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


-RECORD 0-----------------------------------------------------------------------------------------------------------------------------
 projectID | 201711                                                                                                                   
 lemmas    | fuel cell stack assembly diagnostics gebze institute technology actively engage research promote hydrogen energy tech... 
-RECORD 1-----------------------------------------------------------------------------------------------------------------------------
 projectID | 710473                                                                                                                   
 lemmas    | weather snow machine drive renewable energy sources climate change shorten ski season less snow precipitation high av... 
-RECORD 2-----------------------------------------------------------------------------------------------------------------------------
 projectID | 673134                                    

## 4. Save a table with `projectID` and `lemmas` to HDFS

In [18]:
%%time

#Save calculated lemmas to HDFS
dir_parquet = Path("/export/ml4ds/IntelComp/Datalake/CORDIS/20220221/new_parquet")

projects.coalesce(40).write.parquet(
    dir_parquet.joinpath(f"projects_NLP.parquet").as_posix(),
    mode="overwrite",
)

CPU times: user 32.3 ms, sys: 3.68 ms, total: 36 ms
Wall time: 50.4 s


## 5. Optional: Check that the generated table looks OK

In [21]:
%%time

#Test that the saved table is correct
projects = spark.read.parquet('/export/ml4ds/IntelComp/Datalake/CORDIS/20220221/new_parquet/projects_NLP.parquet')
print('Number of lemmatized projects:', projects.count())
projects.show(n=10, truncate=120, vertical=True)

Number of lemmatized projects: 61163
-RECORD 0-----------------------------------------------------------------------------------------------------------------------------
 projectID | 689074                                                                                                                   
 lemmas    | holistic optimisation ship design operation life cycle maritime product typically associate large investment seldom b... 
-RECORD 1-----------------------------------------------------------------------------------------------------------------------------
 projectID | 251681                                                                                                                   
 lemmas    | development nitric oxide releasing stent treatment coronary artery disease coronary artery disease one lead cause dea... 
-RECORD 2-----------------------------------------------------------------------------------------------------------------------------
 projectID | 75605

In [22]:
%%time

projects = spark.read.parquet('/export/ml4ds/IntelComp/Datalake/CORDIS/20220221/new_parquet/projects.parquet')
lemmas = spark.read.parquet('/export/ml4ds/IntelComp/Datalake/CORDIS/20220221/new_parquet/projects_NLP.parquet')

projects_lemmas = (projects.join(lemmas, projects.projectID ==  lemmas., "right")
                      .drop(lemmas.id)
                )

print('Number of projects in joint table:', projects_lemmas.count())
projects_lemmas.show(n=10, truncate=120, vertical=True)

AttributeError: 'DataFrame' object has no attribute 'id'

In [18]:
%%time

#Using SQL Expression
papers_lemmas = papers_lemmas.filter(F.array_contains("fieldsOfStudy", 'Computer Science'))
print('Number of papers in Computer Science:', papers_lemmas.count())
papers_lemmas.show(n=10, truncate=120, vertical=True)

Number of papers in Computer Science: 13654631


-RECORD 0---------------------------------------------------------------------------------------------------------------------------------
 id            | 00004ddfe8089303589fb12cddc05fefc7a0bd96                                                                                 
 fieldsOfStudy | [Computer Science]                                                                                                       
 lemmas        | using static total causal ordering protocols achieve ordered view synchrony view synchronous communication vsc servic... 
-RECORD 1---------------------------------------------------------------------------------------------------------------------------------
 id            | 000051c2d8eff18654e5eaf3e636c02028ef96bb                                                                                 
 fieldsOfStudy | [Computer Science]                                                                                                       
 lemmas        | author thi